# Docker Cheatsheet

**Eric Kerfoot\
School of Biomedical Engineering & Imaging Sciences\
King's College London**


This is a short rundown of the most used Docker commands and common arguments.
Use the `--help` flag with the docker command for more info on the other commands.

Examples: `docker --help`, `docker build --help`, `docker container ls --help`

Useful links:
* Docker docs: https://docs.docker.com/
* Docker command reference: https://docs.docker.com/engine/reference/commandline/cli/
* Dockerfile reference: https://docs.docker.com/engine/reference/builder/ 
* Docker samples: https://docs.docker.com/samples/ https://github.com/docker/labs


## `docker run`

Create and run a container from a named image available locally or from the connected registry. 

Usage: `docker run [OPTIONS] IMAGE[:TAG|@DIGEST] [COMMAND] [ARG...]`

* `IMAGE` has form `[HOSTNAME/][NAMESPACE/]REPOSITORY` where `HOSTNAME` is the address/port of the user registry if used, `NAMESPACE` is the namespace or user under which the repository is found, and `REPOSITORY` is the name under which the image is stored.  

* `TAG` names different versions of images in a repository, default is `latest`.

* `COMMAND` is command to run in container, default is to run command given by `ENTRYPOINT/CMD`.

* `ARG` are further arguments to pass to command.

Common options:
* `-t`: allocate a terminal
* `-i`: interactive mode
* `--rm`: remove container when completed
* `-d`: run container detached, ie. in background
* `--net=host`: connect container's network to host's network
* `-e ENV=VAL`: specify an environment variable set inside the container
* `-p [HOSTADDR:]HOSTPORT:CONTAINERPORT`: map a host port to a container port, eg. `-p 8080:80`
* `-v HOSTDIR:CONTAINERDIR[:PERMS]`: mount host directory as a directory accessible in the container, eg. `-v $HOME/mydata:/data`
* `--gpus all`: mount all host GPUs as devices in the container, can choose devices with "device=" specification, eg. `--gpus \"device=0,1\"`

Example:
```sh
$ docker run erickerfoot/pytorch-test:testtag
1.5.0+cu101 0
```

## `docker build`

Builds an image from a Dockerfile and stores it locally. This will pull images from a connected registry if needed.

Usage: `docker build [OPTIONS] PATH | URL | -`

* `PATH`: directory containing Dockerfile and associated data files, this directory will be copied during the build
* `URL`: location of git repository, targball, or text file to use as build directory

Common options:
* `-f`: specify a Dockerfile, default is `./Dockerfile` in the build directory, can use `-` to read from stdin
* `-t REPONAME:TAG`: tag the built image with the given repository name (eg. `NAME` or `NAMESPACE/NAME`) and tag

Example:

```sh
$ docker build .
Sending build context to Docker daemon  3.072kB
Step 1/3 : from python
 ---> 659f826fabf4
Step 2/3 : COPY hello.py /
 ---> 35343acd247a
Step 3/3 : CMD ["python","hello.py"]
 ---> Running in 732d46910d66
Removing intermediate container 732d46910d66
 ---> 27e884f60275
Successfully built 27e884f60275
```

## `docker image`

View and manipulate stored images.

Usage: `docker image COMMAND`

Common commands:
* `ls`: view images, `-a` to see all 
* `prune`: remove unused images
* `rm`: remove named images, by repository name or image ID
* `pull`: download a named image from a registry
* `push`: upload a named image to a registry
* `save`: save a named image to a tar archive, `-o` to specify file otherwise sent to stdout

Example:
```sh
$ docker image rm redis
Untagged: redis:latest
Untagged: redis@sha256:ec277acf143340fa338f0b1a9b2f23632335d2096940d8e754474e21476eae32
Deleted: sha256:36304d3b4540c5143673b2cefaba583a0426b57c709b5a35363f96a3510058cd
Deleted: sha256:0a0f29e43c3a5555d675d253ff51d73e4d238bd558f11ae9b63d2a2a14251b36
```

## `docker container`

View and manipulate running containers

Usage: `docker container COMMAND`

Common commands:
* `ls`: view containers, `-a` to see all including stopped containers
* `prune`: remove stopped containers
* `stop`: stop running containers
* `rm`: more containers, `--force` to forcibly stop named running containers
* `attach`: attach streams to a container for interaction
* `exec`: execute a command in the named container

Example:
```sh
$ docker container ls -a
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                      PORTS               NAMES
03d7b9923d63        hello-flask         "flask run --host=0.…"   13 seconds ago      Up 12 seconds               5000/tcp            practical_herschel
d87e700df2fb        hello-flask         "flask run --host=0.…"   20 seconds ago      Exited (0) 16 seconds ago                       youthful_moser
```

## `docker kill`
Kill named running containers.

## `docker rm`

Remove named containers, `-f` or `--force` to forcibly stop running containers.

## `docker rmi`
Remove named images, `-f` or `--force` to forcible removal.

## `docker login/logout`
Log into or out of a registry server.

### `docker version`
Generate Docker version information for client and server.

---
## Dockerfile Reference

Dockerfiles are scripts for setting up an image. From a base image a new one is created by installing packages, copying files, and configuring the image as if it were a full Linux environment. This script is composed of a sequence of commands which are run in order, with Docker caching the state of the image between each one, allowing the developer to make changes without having to repeat unchanged steps.

The most-used commands are:

* `FROM`: states the image to base the configuration on, `scratch` for a completely blank image.
* `COPY`: copy file(s) into a location in the image.
* `RUN`: run a command in the image, if given as a string command (eg. `RUN echo Hello run`) the command is run using `/bin/sh`, if in exec form (eg. `RUN ["/bin/echo","Hello run"]`) this is run without shell invocation or interpretation. By default the command is run as root.
* `ENV`: set an environment variable, either as `ENV key value...` or `ENV key=value, key=value, ...`.
* `EXPOSE`: states that the nominated port number is meant to be exposed by the image when run but doesn't ensure this is the case.
* `WORKDIR`: sets the working directory in the image of any subsequent commands.
* `USER`: switch the user executing commands to the nominated username, which must already exist in the image
* `ENTRYPOINT`: state the command to run when the container starts, this can be as a shell command (`ENTRYPOINT echo Hello run`) or exec command (`ENTRYPOINT ["/bin/echo","Hello run"]`). Arguments passed to the container in `docker run`, or those in the `CMD` command if none provided, are appended to the arguments of the command to run.
* `CMD`: like `ENTRYPOINT` however if arguments are passed to `docker run` these must state a new executable to run as the command followed by its arguments. 

Example Dockerfile from https://github.com/docker/labs/blob/master/beginner/flask-app/Dockerfile:

```dockerfile
# our base image
FROM alpine:3.12

# Install python and pip
RUN apk add --update py2-pip

# upgrade pip
RUN pip install --upgrade pip

# install Python modules needed by the Python app
COPY requirements.txt /usr/src/app/
RUN pip install --no-cache-dir -r /usr/src/app/requirements.txt

# copy files required for the app to run
COPY app.py /usr/src/app/
COPY templates/index.html /usr/src/app/templates/

# tell the port number the container should expose
EXPOSE 5000

# run the application
CMD ["python", "/usr/src/app/app.py"]
```